In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='/media/reza/ResearchDrive/Courses/Course_PyTorch',train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='/media/reza/ResearchDrive/Courses/Course_PyTorch',train=False, download=True, transform=transform)

In [3]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [16]:
train_loader

In [4]:
# Hyper parameters
learning_rate = 0.001
sequence_length = 28
hidden_size = 128
num_classes = 10
batch_size = 64
input_size = 28
num_layers = 2
num_epochs = 3

In [6]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)
            
def __len__(self):
    """Number of batches"""
    return len(self.dl)

In [7]:
device = get_default_device()

In [8]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, 
    num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h = torch.zeros(self.num_layers, x.size(0), 
                        self.hidden_size).to(device) 
        c = torch.zeros(self.num_layers, x.size(0), 
                        self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h, c))  
        
        out = self.fc(out[:, -1, :])
        return out

In [9]:
model = RNN(input_size, hidden_size, num_layers, num_classes)
to_device(model, device)

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/3], Step [100/938], Loss: 0.6723
Epoch [1/3], Step [200/938], Loss: 0.4324
Epoch [1/3], Step [300/938], Loss: 0.2338
Epoch [1/3], Step [400/938], Loss: 0.3157
Epoch [1/3], Step [500/938], Loss: 0.2220
Epoch [1/3], Step [600/938], Loss: 0.2879
Epoch [1/3], Step [700/938], Loss: 0.2174
Epoch [1/3], Step [800/938], Loss: 0.2128
Epoch [1/3], Step [900/938], Loss: 0.1313
Epoch [2/3], Step [100/938], Loss: 0.0682
Epoch [2/3], Step [200/938], Loss: 0.2315
Epoch [2/3], Step [300/938], Loss: 0.0133
Epoch [2/3], Step [400/938], Loss: 0.0391
Epoch [2/3], Step [500/938], Loss: 0.1899
Epoch [2/3], Step [600/938], Loss: 0.2500
Epoch [2/3], Step [700/938], Loss: 0.0669
Epoch [2/3], Step [800/938], Loss: 0.0187
Epoch [2/3], Step [900/938], Loss: 0.1639
Epoch [3/3], Step [100/938], Loss: 0.1213
Epoch [3/3], Step [200/938], Loss: 0.0320
Epoch [3/3], Step [300/938], Loss: 0.0149
Epoch [3/3], Step [400/938], Loss: 0.1085
Epoch [3/3], Step [500/938], Loss: 0.1533
Epoch [3/3], Step [600/938], Loss:

In [13]:
# Evaluate the model
model.eval()
with torch.no_grad():
    right = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, 
                 input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        right += (predicted == labels).sum().item()
        
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * right / total))

Test Accuracy of the model on the 10000 test images: 97.72 %


In [15]:
images.shape

torch.Size([16, 28, 28])

In [21]:
labels.shape

torch.Size([16])

In [19]:
predicted

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6])